In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,2,26,2024-07-01 20:14:15
1,大安區,21,4,17,2024-07-01 20:13:21
2,大安區,16,6,10,2024-07-01 20:11:20
3,大安區,11,1,10,2024-07-01 20:12:20
4,大安區,16,1,15,2024-07-01 20:12:20
...,...,...,...,...,...
1424,臺大公館校區,30,0,29,2024-07-01 20:03:15
1425,臺大公館校區,20,1,19,2024-07-01 20:04:22
1426,臺大公館校區,24,8,15,2024-07-01 20:14:20
1427,臺大公館校區,40,28,3,2024-07-01 20:14:20


In [2]:
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1530  156  2255  156
中正區     3209  121  1511  121  1671  121
信義區     3331  106  1260  106  1967  106
內湖區     3499  168  1397  168  2063  168
北投區     2472   94   908   94  1534   94
南港區     2317   92  1110   92  1184   92
士林區     2937  127  1037  127  1825  127
大同區     1629   62   566   62  1049   62
大安區     4987  185  2068  185  2819  185
文山區     2203   97   833   97  1367   97
松山區     2343   88   871   88  1454   88
臺大公館校區  1437   53   536   53   866   53
萬華區     2057   80   982   80  1049   80

In [3]:
#pivot
#stack,unstack
#欄位有名稱
s1 = df1.stack(level=[0,1])
s1.index.names = ['行政區域','主題','數量']
s1.unstack(level=['數量'])

/tmp/ipykernel_27908/4239222652.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df1.stack(level=[0,1])


數量            車輛數  站點數
行政區域   主題             
中山區    可借數量  1530  156
       可還數量  2255  156
       總數量   3819  156
中正區    可借數量  1511  121
       可還數量  1671  121
       總數量   3209  121
信義區    可借數量  1260  106
       可還數量  1967  106
       總數量   3331  106
內湖區    可借數量  1397  168
       可還數量  2063  168
       總數量   3499  168
北投區    可借數量   908   94
       可還數量  1534   94
       總數量   2472   94
南港區    可借數量  1110   92
       可還數量  1184   92
       總數量   2317   92
士林區    可借數量  1037  127
       可還數量  1825  127
       總數量   2937  127
大同區    可借數量   566   62
       可還數量  1049   62
       總數量   1629   62
大安區    可借數量  2068  185
       可還數量  2819  185
       總數量   4987  185
文山區    可借數量   833   97
       可還數量  1367   97
       總數量   2203   97
松山區    可借數量   871   88
       可還數量  1454   88
       總數量   2343   88
臺大公館校區 可借數量   536   53
       可還數量   866   53
       總數量   1437   53
萬華區    可借數量   982   80
       可還數量  1049   80
       總數量   2057   80